# Installing packages

In [ ]:
!pip install lime
!pip install gdown
!pip install -q pandas
!pip install --user -q torch
!pip install -q transformers
!pip install --user -q pytest 
!pip install --user -q tqdm

***Warning***: Depending on the runtime used, you might have to restart the kernel in order for the new libraries to be located properly.

# Import libraries

In [ ]:
from tensorflow import keras
import warnings
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime.submodular_pick
import lime.lime_tabular
import gdown
import itertools
import functools as fu
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
from pathlib import Path
import pickle
import tensorflow as tf
import tensorflow.keras as keras
import torch
import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import AutoModel, AutoTokenizer

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, UpSampling1D
from keras.regularizers import l2, l1, l1_l2

# Define the number of GPU's and batch size you can support


In [ ]:
max_length = 512
num_gpu = 1
batch_size = 30 * num_gpu

# Helper functions

Computing classification probabilities for each class.

In [ ]:
# Converts logits into probabilities of the tweet belonging to each class 
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) 

# Get the predicted class for each tweet, along with the probability of it belonging to the predicted class 
def class_and_confidence(preds):
    softmaxes = []
    for logits in preds[0]:
        sfmax = softmax(logits)
        argmax = logits.argmax()
        softmaxes.append([argmax, sfmax[argmax]])
    return softmaxes

Performing encoding for clean tweets. 

In [ ]:
import torch
from transformers import BertTokenizer, TFBertForSequenceClassification

from transformers import AutoModel, AutoTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

def convert_example_to_feature(review):
  return tokenizer.encode_plus(review,
                add_special_tokens = True, 
                max_length = max_length, 
                pad_to_max_length = True, 
                return_attention_mask = True, 
              )
  

def map_example_to_dict(input_ids, attention_masks, token_type_ids):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }


def encode_tweets(ds):
   input_ids_list = []
   token_type_ids_list = []
   attention_mask_list = []
   label_list = []
   for index, row in tqdm(ds.iterrows()):
     bert_input = convert_example_to_feature(row["tweet"])
     input_ids_list.append(bert_input['input_ids'])
     token_type_ids_list.append(bert_input['token_type_ids'])
     attention_mask_list.append(bert_input['attention_mask'])
     

   return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list)).map(map_example_to_dict)

In [ ]:
# Same as above but tailored for feature visuation.

def map_tweet_to_dict(input_ids, attention_masks, token_type_ids):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }

def encode_tweet(tweet):
   input_id = []
   token_type_id = []
   attention_mask = []
   label = []

   bert_input = convert_example_to_feature(tweet)
   input_id.append(bert_input['input_ids'])
   token_type_id.append(bert_input['token_type_ids'])
   attention_mask.append(bert_input['attention_mask'])

   return tf.data.Dataset.from_tensor_slices((input_id, attention_mask, token_type_id)).map(map_tweet_to_dict)

# Predict proba function needed for feature visualizaiton: Gives probabilities for belonging to each of the two classes.
def predict_proba(tweets):
    result = []
    for tweet in tweets:
        encoded = encode_tweet(tweet)
        batched = encoded.batch(1)
        preds = loaded_model.predict(batched)
        cls_pred = class_and_confidence(preds)

        cls = cls_pred[0][0]
        pred = cls_pred[0][1]

        if cls == 1:
             result.append(np.array([1 - pred,pred]))
        else:
            result.append(np.array([pred,1 - pred]))

            
    return np.array(result)

# Data generation #
Here just 20000 tweets were analyzed due to time constraints and results not much differing. 

In [ ]:
def create_data(fancy_cleaned_tweets):
  df = pd.read_csv(fancy_cleaned_tweets, sep=';', header=0)
  class_samples = 10000
  df2 = df
  df2 = pd.concat([df.iloc[:class_samples], df.iloc[-class_samples:]])
  df2 = df2.dropna()
    
  tweets_encoded = encode_tweets(df2)  

  return tweets_encoded, list(df2['sentiment']), list(df2['tweet']) 

def make_train_data():
    tweets_encoded, sentiments_list, tweets_list =  create_data("tweets_clean_full_min_fancy.csv")
    return tweets_encoded, sentiments_list, tweets_list 

In [ ]:
ds_X_eval, Y_eval, X_eval = make_train_data()

# Load trained model #

In [ ]:
import torch
from transformers import BertTokenizer, TFBertForSequenceClassification

from transformers import AutoModel, AutoTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [ ]:
loaded_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)
loaded_model.load_weights("bert_b30.h5")

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

loaded_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Find tweets with best/worst prediction #

In [ ]:
# Obtains tweets with best prediction belonging to class 1 and 0, along with their true label and probability of beloning to the true label/class
def find_best_preds(trained_model, num_preds):

    ds_X_eval, Y_eval, X_eval = make_train_data()
    encoded = ds_X_eval.batch(batch_size)
    
    preds = trained_model.predict(encoded, batch_size)
    predictions = class_and_confidence(preds)
 
    stored_0 =[];
    stored_1 =[];
    

    for i in range(len(Y_eval)):

        diff = abs(Y_eval[i] - predictions[i][1]);
        
        #Store the current tweet if its prediction is better than that of the last tweet
        if Y_eval[i] ==1 and (len(stored_1)==0 or diff <= stored_1[len(stored_1)-1][1]):
            stored_1.append((i, diff, Y_eval[i], predictions[i][1], X_eval[i]));
            
        if Y_eval[i] ==0 and (len(stored_0)==0 or diff <= stored_0[len(stored_0)-1][1]):
            stored_0.append((i, diff, Y_eval[i], predictions[i], X_eval[i]));

    return stored_1[-num_preds:], stored_0[-num_preds:];
        

In [ ]:
# Same as above but worst prediction
def find_worst_preds(trained_model, num_preds):

    ds_X_eval, Y_eval, X_eval = make_train_data()
    encoded = ds_X_eval.batch(batch_size)
    
    preds = trained_model.predict(encoded, batch_size)
    predictions = class_and_confidence(preds)
 
    stored_0 =[];
    stored_1 =[];
    
    
    for i in range(len(Y_eval)):

        diff = abs(Y_eval[i] - predictions[i][1]);

        if Y_eval[i] ==1 and (len(stored_1)==0 or diff >= stored_1[len(stored_1)-1][1]):
            stored_1.append((i, diff, Y_eval[i], predictions[i][1], X_eval[i]));
            
        if Y_eval[i] ==0 and (len(stored_0)==0 or diff >= stored_0[len(stored_0)-1][1]):
            stored_0.append((i, diff, Y_eval[i], predictions[i], X_eval[i]));

    return stored_1[-num_preds:], stored_0[-num_preds:];

# Obtain the tweets with the best predictions 

In [ ]:
#Contain tweets belonging to class 1 and class 0 with best predictions
c1, c0 = find_best_preds(loaded_model, 5);

In [ ]:
#Contain tweets belonging to class 1 and class 0 with worst predictions
d1, d0 = find_worst_preds(loaded_model, 5);

# Instantiate the explainer #

In [ ]:
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
from lime.lime_text import IndexedString,IndexedCharacters
from lime.lime_base import LimeBase
from sklearn.linear_model import Ridge, lars_path
from lime.lime_text import explanation
from functools import partial
import scipy as sp
from sklearn.utils import check_random_state

explainer = LimeTextExplainer(class_names=[0, 1])

# Compute best/worst explanations for class 1

In [ ]:
instance = 4;

exp = explainer.explain_instance(c1[instance][4], predict_proba, num_features=20, labels=(1,))

exp.show_in_notebook(text=True)
exp.save_to_file('best_1.html')

In [ ]:
instance = 4;

exp = explainer.explain_instance(d1[instance][4], predict_proba, num_features=20, labels=(1,))

exp.show_in_notebook(text=True)
exp.save_to_file('1_worst_1.html')

# Compute best/worst explanations for class 0

In [ ]:
instance = 4;

exp = explainer.explain_instance(c0[instance][4], predict_proba, num_features=20, labels=(0,))

exp.show_in_notebook(text=True)
exp.save_to_file('0_best_1.html')

In [ ]:
instance = 4;

exp = explainer.explain_instance(d0[instance][4], predict_proba, num_features=20, labels=(0,))

exp.show_in_notebook(text=True)
exp.save_to_file('0_worst_1.html')